# FAKE NEWS DETECTION

The following notebook contains all relevant computations and investigations for the exam in DS821: News and Market Sentiment Analysis. The structure is predominantly corresponding to the structure of the report. However, most section is labeled with the number corresponding to the section of the report for transparency.  

In [ ]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import math
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_validate
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity

## 0. DATA LOAD AND INSPECTION

In [3]:
# load the data
fake = pd.read_csv("/Users/FrederikkeB/Dropbox/DS 3/News Analysis/Exam/data/Fake.csv")
true = pd.read_csv("/Users/FrederikkeB/Dropbox/DS 3/News Analysis/Exam/data/True.csv")

In [4]:
# check dataframe
fake.head(2)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"


In [5]:
# check dataframe
true.head(2)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"


In [6]:
# assign class labels to each dataframe
true["label"] = 0
fake["label"] = 1

In [7]:
# merge to two dataframes together 
df = pd.concat([fake, true], ignore_index=True)

In [8]:
# inspect data frame
print("Shape:", df.shape)
print(df["label"].value_counts())
print(df.isna().sum())

Shape: (44898, 5)
label
1    23481
0    21417
Name: count, dtype: int64
title      0
text       0
subject    0
date       0
label      0
dtype: int64


## 3. DATA CLEANING

In [9]:
# check for duplicates
df.duplicated(subset=["text"]).sum()

np.int64(6252)

In [10]:
# remove duplicates 
len_before = len(df)
df = df.drop_duplicates(subset=["text"], keep="first")
len_after = len(df)

print("Duplicates removed:", len_before - len_after)


Duplicates removed: 6252


In [11]:
# inspect how many articles comes from reuters
df["starts_with_reuters"] = df["text"].str.lower().str.contains(
    r"^.*\(\s*reuters\s*\)\s*-", regex=True, na=False
)
# divide by class
df.groupby("label")["starts_with_reuters"].mean()


label
0    0.992025
1    0.000000
Name: starts_with_reuters, dtype: float64

In [12]:
# remove potential metadata in the beginning of each body text
def remove_leading_metadata(text):
    pattern = r"^[A-Z\s\/]+\s*\([A-Za-z]+\)\s*[–—-]\s+"
    return re.sub(pattern, "", text).strip()

# run on text column
df["text_clean"] = df["text"].apply(remove_leading_metadata)

In [ ]:
# sanity check
df["metadata_removed"] = df["text"] != df["text_clean"]
df.groupby("label")["metadata_removed"].mean()

label
0    0.992733
1    0.214437
Name: metadata_removed, dtype: float64

In [ ]:
# examine the occurence of url in articles
df["url_count"] = df["text"].str.count(r"http[s]?://")

# print the difference between classes
df.groupby("label")["url_count"].mean()


label
0    0.000000
1    0.208823
Name: url_count, dtype: float64

In [ ]:
# inspect urls context in articles
def url_context(text, window=5):
    tokens = text.split()
    contexts = []
    for i, token in enumerate(tokens):
        if token.startswith("http"):
            start = max(i - window, 0)
            end = min(i + window + 1, len(tokens))
            contexts.append(" ".join(tokens[start:end]))
    return contexts

df["url_contexts"] = df["text"].apply(url_context)

# print observations including urls
fake_contexts = df[df["label"] == 1]["url_contexts"].explode().dropna()
fake_contexts.head(10)


2     Clarke, email search warrant filed https://t.c...
3     by July 24 next year. https://t.co/Fg7VacxRtJ ...
3     of an internal server error: https://t.co/zrWp...
3     appears to be on all https://t.co/dkhw0AlHB4 p...
3     2017It s also all over https://t.co/ayBlGmk65Z...
16    this to the Haunted Mansion? https://t.co/XrOv...
16    looking as a fucking Pokemon. https://t.co/HFY...
16    the future of the planet. https://t.co/65FhbQH...
19    don t. You don t https://t.co/7lHYkIloyz Senat...
19    to clean house of partisans https://t.co/g8Swg...
Name: url_contexts, dtype: object

In [16]:
# remove web related entities from body text 
def remove_urls_and_html(text):
    text = re.sub(r"http\S+|www\S+", " ", text)   
    text = re.sub(r"&\w+;", " ", text)            
    return text

df["text_clean"] = df["text_clean"].apply(remove_urls_and_html)

In [ ]:
# define web specific noise
noise = {
    "https", "http", "www", "amp", "quot", "cdata", "js",
    "pic", "youtu", "flickr", "getty", "wikimedia",
    "screenshot", "src", "createelement", "getelementbyid",
    "getelementsbytagname", "parentnode", "insertbefore",
    "jssdk", "xfbml", "filessupport", "21wire", 
}

# translate to regex
noise_pattern = re.compile(
    r"\b(" + "|".join(map(re.escape, noise)) + r")\b",
    flags=re.IGNORECASE
)

# remove noise
def remove_noise(text):
    return noise_pattern.sub("", text)

# apply to cleaned text
df["text_clean"] = df["text_clean"].apply(remove_noise)

In [18]:
# lowercase body text
df["text_clean"] = df["text_clean"].str.lower()

## 3. TOKENIZATION

In [19]:
# load stopwords
stopwords = set(ENGLISH_STOP_WORDS)

In [ ]:
# function combining each tokenization step
def tokenize(text):

    # extract alphabetic tokens
    tokens = re.findall(r"[a-zA-Z]+", text)

    # remove stopwords
    tokens = [t for t in tokens if t not in stopwords]

    # remove tokens with less than 2 characters
    tokens = [t for t in tokens if len(t) > 2]

    return tokens

# apply to cleaned text 
df["tokens"] = df["text_clean"].apply(tokenize)

In [ ]:
# inspect clean dataframe
df.head(2)

,title,text,subject,date,label,starts_with_reuters,text_clean,metadata_removed,url_count,url_contexts,tokens
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1,False,donald trump just couldn t wish all americans ...,False,0,[],"[donald, trump, just, couldn, wish, americans,..."
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1,False,house intelligence committee chairman devin nu...,False,0,[],"[house, intelligence, committee, chairman, dev..."


## 4. EXPLORATORY DATA ANALYSIS

### 4.1 LENGTH AND STYLE DIFFERENCES

In [21]:
# check basic properties
print("Dataset shape:", df.shape)
print("\nClass distribution:")
print(df["label"].value_counts())

print("\nClass proportions:")
print(df["label"].value_counts(normalize=True))

Dataset shape: (38591, 11)

Class distribution:
label
0    21191
1    17400
Name: count, dtype: int64

Class proportions:
label
0    0.549118
1    0.450882
Name: proportion, dtype: float64


In [ ]:
# basic length features
df["char_count"] = df["text_clean"].str.len()
df["word_count"] = df["text_clean"].str.split().str.len()

# summary statistics per class
length_stats = df.groupby("label")[["char_count", "word_count"]].agg(
    ["mean", "median", "std"]
)

# print results
length_stats.index = ["True", "Fake"]
length_stats

char_count                       word_count                   
             mean  median          std        mean median         std
True  2359.007645  2197.0  1684.412767  382.210655  356.0  273.922864
Fake  2550.922299  2233.0  2195.548446  426.502759  377.0  355.087980

In [ ]:
# function extracting most used words in each class
def top_words_from_tokens(token_series, n=100):
    tokens = [t for tokens in token_series for t in tokens]
    return Counter(tokens).most_common(n)

# apply function and store in each list
top_true = top_words_from_tokens(df[df["label"] == 0]["tokens"], 100)
top_fake = top_words_from_tokens(df[df["label"] == 1]["tokens"], 100)

# print
print("Top words – True news:")
print(top_true)

print("\nTop words – Fake news:")
print(top_fake)

Top words – True news:
[('said', 97814), ('trump', 54066), ('president', 27857), ('state', 20781), ('government', 18550), ('house', 16462), ('states', 16412), ('republican', 16135), ('new', 15775), ('united', 15159), ('people', 15076), ('year', 14583), ('told', 14061), ('party', 12589), ('election', 12171), ('reuters', 10897), ('campaign', 10534), ('donald', 10370), ('security', 10006), ('percent', 9920), ('north', 9696), ('clinton', 9464), ('white', 9452), ('court', 9356), ('obama', 9330), ('senate', 9161), ('country', 8779), ('china', 8714), ('minister', 8544), ('officials', 8364), ('week', 8333), ('democratic', 8319), ('tuesday', 8189), ('foreign', 8170), ('national', 8139), ('law', 8133), ('administration', 8100), ('tax', 8063), ('including', 8010), ('presidential', 7983), ('military', 7980), ('russia', 7947), ('wednesday', 7861), ('years', 7744), ('political', 7627), ('thursday', 7564), ('statement', 7490), ('did', 7378), ('time', 7347), ('friday', 7257), ('support', 7092), ('kore

### 4.2 SENTITMENT

#### FINBERT SENTIMENT

In [ ]:
# define FinBERT-model
finbert_model = pipeline(
    "sentiment-analysis",
    model="ProsusAI/finbert",
    device="mps",
    truncation=True,
    max_length=512
)

Device set to use mps


In [ ]:
# function extracting scores from model
def finbert_score(text):
    if not isinstance(text, str) or not text.strip():
        return np.nan   

    out = finbert_model(text)[0]
    label = out["label"]
    score = out["score"]

    if label == "positive":
        return score
    elif label == "negative":
        return -score
    else:  # neutral
        return 0.0

In [46]:
# translate scores to labels
def finbert_label(score, eps=0.1):
    if pd.isna(score):
        return np.nan
    if score > eps:
        return "positive"
    if score < -eps:
        return "negative"
    return "neutral"


In [ ]:
# define a subset  
finbert_subset = (
    df
    .groupby("label", group_keys=False)
    .apply(lambda x: x.sample(n=2000, random_state=42))
)

# apply model to subset 
finbert_subset["finbert_score"] = finbert_subset["text_clean"].apply(finbert_score)
finbert_subset["finbert_label"] = finbert_subset["finbert_score"].apply(
    lambda x: finbert_label(x, eps=0.1)
)

/var/folders/qw/cnk8h6ld5nj287lndrx4g8kh0000gn/T/ipykernel_63248/124310353.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=2000, random_state=42))


In [ ]:
# print results
finbert_subset["finbert_label"].value_counts(normalize=True)

finbert_label
neutral     0.507762
negative    0.459940
positive    0.032298
Name: proportion, dtype: float64

In [ ]:
# print results by class 
finbert_by_class = (
    finbert_subset
    .groupby("label")["finbert_label"]
    .value_counts(normalize=True)
    .unstack()
)

finbert_by_class.index = ["True news", "Fake news"]
finbert_by_class


finbert_label,negative,neutral,positive
True news,0.610000,0.330000,0.060000
Fake news,0.309428,0.686058,0.004514


#### ROBERTA SENTIMENT

In [ ]:
# define RoBERTa-model 
roberta_model = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    device="mps",
    truncation=True,
    max_length=512
)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps


In [ ]:
# function extracting scores from model
def roberta_score(text):
    if not isinstance(text, str) or not text.strip():
        return np.nan   

    out = roberta_model(text)[0]
    label = out["label"]
    score = out["score"]

    if label == "positive":
        return score
    elif label == "negative":
        return -score
    else:  # neutral
        return 0.0

In [ ]:
# translate scores to labels
def roberta_label(score, eps=0.1):
    if pd.isna(score):
        return np.nan
    if score > eps:
        return "positive"
    if score < -eps:
        return "negative"
    return "neutral"


In [ ]:
# define a subset
roberta_subset = (
    df
    .groupby("label", group_keys=False)
    .apply(lambda x: x.sample(n=2000, random_state=42))
)

# apply model to subset
roberta_subset["roberta_score"] = roberta_subset["text_clean"].apply(roberta_score)
roberta_subset["roberta_label"] = roberta_subset["roberta_score"].apply(
    lambda x: roberta_label(x, eps=0.1)
)

/var/folders/qw/cnk8h6ld5nj287lndrx4g8kh0000gn/T/ipykernel_63248/521462801.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=2000, random_state=42))


In [ ]:
# print results
roberta_subset["roberta_label"].value_counts(normalize=True)

roberta_label
neutral     0.586880
negative    0.365048
positive    0.048072
Name: proportion, dtype: float64

In [ ]:
# print results by class 
roberta_by_class = (
    roberta_subset
    .groupby("label")["roberta_label"]
    .value_counts(normalize=True)
    .unstack()
)

roberta_by_class.index = ["True news", "Fake news"]
roberta_by_class


roberta_label,negative,neutral,positive
True news,0.154500,0.810000,0.035500
Fake news,0.576229,0.363089,0.060682


### 4.3 VOCABULARY CONTRASTS

In [ ]:
# function counting tokens in a series of token lists
def count_tokens(token_series):
    return Counter([t for tokens in token_series for t in tokens])


# token counts per class
true_counts = count_tokens(df[df["label"] == 0]["tokens"])
fake_counts = count_tokens(df[df["label"] == 1]["tokens"])

min_freq = 50  


# define a shared vocabulary of tokens appearing sufficiently often in both classes
vocab = {
    w for w in fake_counts
    if fake_counts[w] >= min_freq and true_counts.get(w, 0) >= min_freq
}


# find words more characteristic of fake news
rel_fake = {
    w: math.log((fake_counts[w] + 1) / (true_counts[w] + 1))
    for w in vocab
}
# sort values
top_fake = sorted(
    rel_fake.items(),
    key=lambda x: x[1],
    reverse=True
)[:50]

print("Words more characteristic of Fake news:\n")
for w, s in top_fake:
    print(f"{w:<15} {s:.2f}")


# find words more characteristic of true news
rel_true = {
    w: math.log((true_counts[w] + 1) / (fake_counts[w] + 1))
    for w in vocab
}
# sort values
top_true = sorted(
    rel_true.items(),
    key=lambda x: x[1],
    reverse=True
)[:50]

print("\nWords more characteristic of True news:\n")
for w, s in top_true:
    print(f"{w:<15} {s:.2f}")

Words more characteristic of Fake news:

featured        4.43
gop             3.72
com             3.44
image           3.26
rep             3.13
screen          3.07
literally       3.04
youtube         2.90
images          2.89
wire            2.87
realdonaldtrump 2.78
stupid          2.57
racist          2.54
wonder          2.48
isn             2.35
campus          2.29
lie             2.26
interesting     2.25
wasn            2.22
mrs             2.22
kids            2.22
racism          2.19
hate            2.17
sounds          2.15
breitbart       2.15
perfectly       2.14
apparently      2.14
joke            2.14
couldn          2.13
guy             2.12
watch           2.12
narrative       2.09
hell            2.06
truth           2.05
explained       2.05
ridiculous      2.04
fun             2.03
actually        2.01
crazy           1.99
video           1.98
remember        1.97
reads           1.97
soros           1.97
bigotry         1.96
clip            1.96
perfect       

### TOPIC MODELLING: LDA 

In [51]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models import CoherenceModel

In [72]:
df_true = df[df["label"] == 0]
df_fake = df[df["label"] == 1]

In [73]:
def run_lda(token_lists, num_topics=7, no_below=20, no_above=0.9):
    
    # Create dictionary
    dictionary = Dictionary(token_lists)
    dictionary.filter_extremes(no_below=no_below, no_above=no_above)
    
    # Create BoW corpus
    corpus_bow = [dictionary.doc2bow(doc) for doc in token_lists]
    
    # Train LDA
    lda = LdaModel(
        corpus=corpus_bow,
        id2word=dictionary,
        num_topics=num_topics,
        passes=10,
        random_state=42
    )
    
    return lda, dictionary, corpus_bow

In [74]:
lda_true, dict_true, corpus_true = run_lda(
    df_true["tokens"].tolist(),
    num_topics=7
)

lda_fake, dict_fake, corpus_fake = run_lda(
    df_fake["tokens"].tolist(),
    num_topics=7
)


In [ ]:
print("Topics – True news")
for t in lda_true.print_topics(num_words=10):
    print(t)

print("\nTopics – Fake news")
for t in lda_fake.print_topics(num_words=10):
    print(t)


Topics – True news
(0, '0.023*"party" + 0.018*"election" + 0.012*"vote" + 0.012*"republican" + 0.011*"trump" + 0.011*"percent" + 0.008*"democratic" + 0.008*"clinton" + 0.008*"presidential" + 0.007*"voters"')
(1, '0.013*"state" + 0.011*"military" + 0.010*"islamic" + 0.010*"forces" + 0.009*"government" + 0.008*"syria" + 0.008*"turkey" + 0.007*"iraq" + 0.007*"saudi" + 0.007*"killed"')
(2, '0.011*"court" + 0.011*"police" + 0.010*"people" + 0.007*"myanmar" + 0.007*"rights" + 0.007*"government" + 0.006*"rohingya" + 0.005*"state" + 0.005*"year" + 0.005*"law"')
(3, '0.045*"trump" + 0.016*"president" + 0.010*"house" + 0.009*"white" + 0.008*"campaign" + 0.007*"donald" + 0.007*"russia" + 0.007*"russian" + 0.005*"department" + 0.005*"committee"')
(4, '0.010*"tax" + 0.008*"percent" + 0.008*"billion" + 0.007*"government" + 0.007*"house" + 0.007*"year" + 0.006*"million" + 0.006*"budget" + 0.006*"new" + 0.006*"companies"')
(5, '0.011*"north" + 0.009*"minister" + 0.009*"korea" + 0.008*"china" + 0.008*"

In [ ]:
# get coherence score for each class
coh_true = CoherenceModel(
    model=lda_true,
    texts=df_true["tokens"].tolist(),
    dictionary=dict_true,
    coherence="c_v"
).get_coherence()

coh_fake = CoherenceModel(
    model=lda_fake,
    texts=df_fake["tokens"].tolist(),
    dictionary=dict_fake,
    coherence="c_v"
).get_coherence()

coh_true, coh_fake


(np.float64(0.4828385696300357), np.float64(0.4477192467662035))

## 5. CLASSIFICATION

In [ ]:
# prepare data for all classification models
X = df["text_clean"]
y = df["label"]

# split traning and test set
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

### 5.1 NAÏVE BAYES

In [ ]:
# define nb-classifier
nb_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        stop_words="english",
        max_df=0.9,
        min_df=50
    )),
    ("nb", MultinomialNB())
])

# define cross validation
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)


In [ ]:
# define scoring metrics
scoring = {
    "accuracy": "accuracy",
    "precision": "precision",
    "recall": "recall",
    "f1": "f1"
}
# run cross validation
cv_results = cross_validate(
    nb_pipeline,
    X_train,
    y_train,
    cv=cv,
    scoring=scoring,
    return_train_score=True
)


In [ ]:
# see results for cross validation
cv_df = pd.DataFrame(cv_results)
cv_df.mean()

fit_time           3.379223
score_time         0.618406
test_accuracy      0.927837
train_accuracy     0.931977
test_precision     0.920231
train_precision    0.924411
test_recall        0.920009
train_recall       0.925039
test_f1            0.920113
train_f1           0.924725
dtype: float64

In [ ]:
# train on whole training set
nb_pipeline.fit(X_train, y_train)

# run predictions on test set
y_test_pred = nb_pipeline.predict(X_test)

# print results
print(classification_report(
    y_test,
    y_test_pred,
    target_names=["True news", "Fake news"]
))

              precision    recall  f1-score   support

   True news       0.93      0.94      0.93      4239
   Fake news       0.92      0.91      0.92      3491

    accuracy                           0.93      7730
   macro avg       0.93      0.92      0.93      7730
weighted avg       0.93      0.93      0.93      7730



#### EXAMINE IMPORTANT WORDS FOR NB

In [ ]:
# get components from pipeline
vectorizer = nb_pipeline.named_steps["tfidf"]
nb_model = nb_pipeline.named_steps["nb"]

# extract words
feature_names = np.array(vectorizer.get_feature_names_out())

Top words indicating Fake news:

              log_odds_fake_vs_true
featured                   4.321071
gop                        3.731271
com                        3.533362
image                      3.490896
acr                        3.480959
hannity                    3.451007
fjs                        3.450252
bundy                      3.395774
screengrab                 3.388768
somodevilla                3.386792
hilarious                  3.382441
boiler                     3.381462
cops                       3.371698
maher                      3.349387
shit                       3.207256
2017the                    3.200610
pundit                     3.128580
screen                     3.111864
literally                  3.106908
reilly                     3.086505
images                     3.083379
ck                         3.075353
ass                        3.070105
disgusting                 3.035628
angerer                    3.030297
mcnamee                    2.97

In [ ]:
# apply log probabilities per class
log_probs = nb_model.feature_log_prob_

# create dataframe for results
df_log_probs = pd.DataFrame(
    log_probs.T,
    index=feature_names,
    columns=["True news", "Fake news"]
)

In [ ]:
# compute the difference between classes
df_log_probs["log_odds_fake_vs_true"] = (
    df_log_probs["Fake news"] - df_log_probs["True news"]
)

In [ ]:
# top words for fake news
top_fake_nb = (
    df_log_probs
    .sort_values("log_odds_fake_vs_true", ascending=False)
    .head(50)
)

# top words for true news
top_true_nb = (
    df_log_probs
    .sort_values("log_odds_fake_vs_true", ascending=True)
    .head(50)
)

print("Top words indicating Fake news:\n")
print(top_fake_nb[["log_odds_fake_vs_true"]])

print("\nTop words indicating True news:\n")
print(top_true_nb[["log_odds_fake_vs_true"]])

### 5.2 CENTROID-BASED CLASSIFIER

In [ ]:
# load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# embed training data
X_train_embeddings = embedding_model.encode(
    X_train.tolist(),
    show_progress_bar=True
)
# convert targets to array
y_train_array = y_train.values


Batches:   0%|          | 0/967 [00:00<?, ?it/s]

In [24]:
# compute class-centroids
true_centroid = X_train_embeddings[y_train_array == 0].mean(axis=0)
fake_centroid = X_train_embeddings[y_train_array == 1].mean(axis=0)

# reshape for cosine similarity
true_centroid = true_centroid.reshape(1, -1)
fake_centroid = fake_centroid.reshape(1, -1)


In [25]:
# embed test data
X_test_embeddings = embedding_model.encode(
    X_test.tolist(),
    show_progress_bar=True
)

Batches:   0%|          | 0/242 [00:00<?, ?it/s]

In [26]:
# define classification logic
def centroid_predict(embeddings, true_centroid, fake_centroid):
    sim_true = cosine_similarity(embeddings, true_centroid).flatten()
    sim_fake = cosine_similarity(embeddings, fake_centroid).flatten()
    
    # predict class with highest similarity
    return np.where(sim_fake > sim_true, 1, 0)

# predict on test set 
y_pred_centroid = centroid_predict(
    X_test_embeddings,
    true_centroid,
    fake_centroid
)

In [27]:
# print results
print(classification_report(
    y_test,
    y_pred_centroid,
    target_names=["True news", "Fake news"]
))

              precision    recall  f1-score   support

   True news       0.88      0.81      0.84      4239
   Fake news       0.79      0.87      0.83      3491

    accuracy                           0.84      7730
   macro avg       0.84      0.84      0.84      7730
weighted avg       0.84      0.84      0.84      7730



In [28]:
# investigate distance between class centroids 
centroid_similarity = cosine_similarity(true_centroid, fake_centroid)[0, 0]
print("Cosine similarity between class centroids:", centroid_similarity)

Cosine similarity between class centroids: 0.83728975


In [ ]:
# cosine similarity to centroids for test set
sim_true = cosine_similarity(X_test_embeddings, true_centroid).flatten()
sim_fake = cosine_similarity(X_test_embeddings, fake_centroid).flatten()

analysis_df = pd.DataFrame({
    "text": X_test.values,
    "true_label": y_test.values,
    "sim_true": sim_true,
    "sim_fake": sim_fake,
    "pred_label": y_pred_centroid
})

# compute margin to both classes for each observation 
analysis_df["margin"] = analysis_df["sim_fake"] - analysis_df["sim_true"]


In [40]:
# very confident fake predictions
analysis_df.sort_values("margin", ascending=False).head(5)

,text,true_label,sim_true,sim_fake,pred_label,margin
2874,the audio of hillary clinton laughing about ge...,1,0.323283,0.579262,1,0.255979
2154,as much as he and his supporters want to say t...,1,0.438703,0.687600,1,0.248897
6638,something that donald trump needs to realize i...,1,0.407673,0.654227,1,0.246554
2573,tomi lahren literally thinks the beating of a ...,1,0.305683,0.551805,1,0.246121
3538,when donald trump isn t getting blasted for hi...,1,0.418764,0.659268,1,0.240503


In [ ]:
# very confident true predictions
analysis_df.sort_values("margin", ascending=True).head(5)

,text,true_label,sim_true,sim_fake,pred_label,margin
5204,iran is fulfilling its commitments under the n...,0,0.522831,0.273593,0,-0.249238
5854,turkey will take the resolution calling on the...,0,0.477216,0.229957,0,-0.247259
485,the united states could shortly broaden talks ...,0,0.525830,0.281796,0,-0.244034
5316,libyan factions involved in u.n.-brokered peac...,0,0.500797,0.261284,0,-0.239513
2927,china has proposed a three-phase plan for reso...,0,0.411447,0.172409,0,-0.239038


In [ ]:
# misclassified articles
analysis_df[analysis_df["true_label"] != analysis_df["pred_label"]].head(5)

,text,true_label,sim_true,sim_fake,pred_label,margin
4,republican presidential front-runner donald tr...,0,0.463641,0.468088,1,0.004446
8,donald trump brought his message of walls and ...,0,0.493613,0.535861,1,0.042248
13,when hillary clinton first ran for president i...,0,0.427209,0.522503,1,0.095294
15,admitting somalis who d been settled for years...,1,0.292669,0.180809,0,-0.111860
18,like the soldiers of oden vigilante group we r...,1,0.352111,0.294505,0,-0.057605
24,a top aide to u.s. president donald trump on s...,0,0.567558,0.654247,1,0.086689
27,the hill released controversial comments sore ...,1,0.624234,0.557003,0,-0.067232
31,the following statements were posted to the ve...,0,0.520369,0.535069,1,0.014700
35,british police said they have evacuated and ar...,0,0.067385,0.068492,1,0.001106
36,south korean police have arrested the owner an...,0,0.240778,0.267489,1,0.026711


### 5.3 ZERO-SHOT 

In [ ]:
# load zero-shot classifier
zero_shot = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device="mps" 
)

Device set to use mps


In [ ]:
# define labels 
candidate_labels = ["fake news", "real news"]

# extract subset
subset_size = 500

X_test_subset = X_test.sample(
    n=subset_size,
    random_state=42
)

y_test_subset = y_test.loc[X_test_subset.index]


In [ ]:
#  define decision logic
def zero_shot_predict(texts):
    outputs = zero_shot(texts, candidate_labels)
    return np.array([
        1 if o["labels"][0] == "fake news" else 0
        for o in outputs
    ])

y_pred_zs = zero_shot_predict(X_test_subset.tolist())


In [ ]:
# print results
print(classification_report(
    y_test_subset,
    y_pred_zs,
    target_names=["True news", "Fake news"]
))

              precision    recall  f1-score   support

   True news       0.39      0.24      0.30       287
   Fake news       0.33      0.50      0.40       213

    accuracy                           0.35       500
   macro avg       0.36      0.37      0.35       500
weighted avg       0.36      0.35      0.34       500

